In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mping
import ast
import math
import shutil

In [ ]:
def get_project_root():
    """Returns project root folder."""
    try:
        current_dir = os.path.dirname(os.path.abspath(__file__))
        print(current_dir)
        return os.path.dirname(current_dir)
    except NameError:
        return os.path.abspath(os.getcwd())

BASE_DIR = get_project_root()

In [ ]:
def delete_files_in_record(directory):
    # Delete all files and subfolder in the recorded directory. 
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try: 
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason {e}')

def clear_files_in_folder():
    folders = ['plot_reward_compare_dqn_ppo', 'plot_coor', 'plot_state']
    
    for folder in folders:
        clear_dir = os.path.join(BASE_DIR, folder)
        if os.path.exists(clear_dir):
            delete_files_in_record(clear_dir)
            print(f"All files in {clear_dir} are deleted")
        else:
            print(f"The directory {clear_dir} does not exist")

clear_files_in_folder()


In [22]:
# input_dir_rewards_dqn = os.path.join(BASE_DIR, '\..' , 'dqn_env', 'test_result', 'reward')
input_dir_rewards_dqn = os.path.join(os.path.dirname(BASE_DIR), 'dqn_env', 'test_result', 'reward')
input_dir_rewards_ppo = os.path.join(os.path.dirname(BASE_DIR), 'ppo_env', 'test_result', 'reward')
input_file = BASE_DIR

filename = f'reward_plot.csv'
filepath_dqn = os.path.join(input_dir_rewards_dqn, filename)
filepath_ppo = os.path.join(input_dir_rewards_ppo, filename)

output_dir = os.path.join(input_file, "plot_reward_compare_dqn_ppo")
os.makedirs(output_dir, exist_ok=True)


df_dqn = pd.read_csv(filepath_dqn)
df_ppo = pd.read_csv(filepath_ppo)

window_size = 500

plt.figure(figsize=(8,6))
plt.plot(df_dqn['Rewards'].rolling(window=window_size).mean(), color = 'blue', marker='o', markersize='2', label = 'DQN')
plt.plot(df_ppo['Rewards'].rolling(window=window_size).mean(), color = 'red', marker='+', markersize='2', label = 'PPO')

leg = plt.legend()
for line in leg.get_lines():
    line.set_linewidth(1)
for text in leg.get_texts():
    text.set_fontsize('large')

plt.xlabel('Reward')
plt.xlabel('Episode')
plt.title('REWARDS')
plt.savefig(os.path.join(output_dir,f'reward_plot.png'))
plt.clf()
plt.close()

In [20]:
input_dir_coor_dqn = os.path.join(os.path.dirname(BASE_DIR), 'dqn_env', 'test_result', 'coordinates')
input_dir_coor_ppo = os.path.join(os.path.dirname(BASE_DIR), 'ppo_env', 'test_result', 'coordinates')

input_file = BASE_DIR

output_dir = os.path.join(input_file, "plot_coor")

num_files_dqn = len([name for name in os.listdir(input_dir_coor_dqn) if os.path.isfile(os.path.join(input_dir_coor_dqn, name))])
num_files_ppo = len([name for name in os.listdir(input_dir_coor_ppo) if os.path.isfile(os.path.join(input_dir_coor_ppo, name))])

if (num_files_dqn != num_files_ppo):
    print("The files is not equal, Please retest to equal episode.")
else:
    filename = f'coordinates{num_files_dqn}_plot.csv'
    num_files = num_files_dqn
    filepath_dqn = os.path.join(input_dir_coor_dqn, filename)
    filepath_ppo = os.path.join(input_dir_coor_ppo, filename)
    os.makedirs(output_dir, exist_ok=True)

    image_path = os.path.join(input_file, 'images', "4_wheeled_robot_testing_obj_pes2.png")
    img = mping.imread(image_path)

    def string_to_floats(s):
        try:
            # Safely evaluate the string as a Python literal
            value_list = ast.literal_eval(s)
            # Convert to floats and return as a Series
            return pd.Series([float(value_list[0]), float(value_list[1]), float(value_list[2]), float(value_list[3])])
        except:
            # Return NaN values if there's an error
            return pd.Series([float('nan'), float('nan'), float('nan'), float('nan')])

    df_dqn = pd.read_csv(filepath_dqn)
    df_ppo = pd.read_csv(filepath_ppo)

    df_dqn[['x_robot','y_robot','x_pedes','y_pedes']] = df_dqn['x_y'].apply(string_to_floats)
    df_ppo[['x_robot','y_robot','x_pedes','y_pedes']] = df_ppo['x_y'].apply(string_to_floats)

    x_dqn_ro = df_dqn['x_robot']
    y_dqn_ro = df_dqn['y_robot']
    x_dqn_pe = df_dqn['x_pedes']
    y_dqn_pe = df_dqn['y_pedes']

    x_ppo_ro = df_ppo['x_robot']
    y_ppo_ro = df_ppo['y_robot']
    x_ppo_pe = df_ppo['x_pedes']
    y_ppo_pe = df_ppo['y_pedes']

    df_dqn['x_ro_path'] = (((x_dqn_ro - 0.85)/(3.58 - 0.85) * (1000 - 153)) + 153)
    df_dqn['y_ro_path'] = (((y_dqn_ro - (-0.179))/(-0.3 - (-0.179)) * (268 - 227)) + 227)
    df_dqn['x_pe_path'] = 741
    df_dqn['y_pe_path'] = (((y_dqn_pe - 0.21)/(-0.3 - 0.21) * (288 - 126)) + 126)

    df_ppo['x_ro_path'] = (((x_ppo_ro - 0.85)/(3.58 - 0.85) * (1000 - 153)) + 153)
    df_ppo['y_ro_path'] = (((y_ppo_ro - (-0.179))/(-0.3 - (-0.179)) * (268 - 227)) + 227)
    df_ppo['x_pe_path'] = 741
    df_ppo['y_pe_path'] = (((y_ppo_pe - 0.21)/(-0.3 - 0.21) * (288 - 126)) + 126)

    x_dqn_ro_path = df_dqn['x_ro_path'].astype(int)
    y_dqn_ro_path = df_dqn['y_ro_path'].astype(int)
    x_dqn_pe_path = df_dqn['x_pe_path'].astype(int)
    y_dqn_pe_path = df_dqn['y_pe_path'].astype(int)

    x_ppo_ro_path = df_ppo['x_ro_path'].astype(int)
    y_ppo_ro_path = df_ppo['y_ro_path'].astype(int)
    x_ppo_pe_path = df_ppo['x_pe_path'].astype(int)
    y_ppo_pe_path = df_ppo['y_pe_path'].astype(int)

    fig,(ax1,ax2) = plt.subplots(2,figsize = (16,8))

    ax1.imshow(img)
    ax1.set_title("DQN")
    ax1.plot(x_dqn_ro_path.values, y_dqn_ro_path.values, 'tab:blue', linewidth=4.0)
    ax1.plot(x_dqn_pe_path.values, y_dqn_pe_path.values, 'tab:red', linewidth=4.0)
    ax1.axis('off')

    ax2.imshow(img)
    ax2.set_title("PPO")
    ax2.plot(x_ppo_ro_path.values, y_ppo_ro_path.values, 'tab:blue', linewidth=4.0)
    ax2.plot(x_ppo_pe_path.values, y_ppo_pe_path.values, 'tab:red', linewidth=4.0)
    ax2.axis('off')

    plt.savefig(os.path.join(output_dir,f'coordinates_dqn_ppo_{num_files}_episode_plot.png'))
    plt.clf()
    plt.close()



In [ ]:
# input_dir_state_ppo = r"/home/chalumpol/Desktop/Test_webots/Second_secenario_pes/ppo_env/my_first_simulation/test_result/state"
# input_dir_state_dqn = r"/home/chalumpol/Desktop/Test_webots/Second_secenario_pes/dqn_env/test_result/state"
input_dir_state_dqn = os.path.join(os.path.dirname(BASE_DIR), 'dqn_env', 'test_result', 'state')
input_dir_state_ppo = os.path.join(os.path.dirname(BASE_DIR), 'ppo_env', 'test_result', 'state')

input_file = BASE_DIR

output_dir = os.path.join(input_file, "plot_state")
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def string_to_floats(s):
    try:
        # Safely evaluate the string as a Python literal
        value_list = ast.literal_eval(s)
        # Convert to floats and return as a Series
        return pd.Series([float(value_list[0]), float(value_list[1]), float(value_list[2]), float(value_list[3]), float(value_list[4]), float(value_list[5]), float(value_list[6]), float(value_list[7]), float(value_list[8]), float(value_list[9]), float(value_list[10]), float(value_list[11]), float(value_list[12]),  float(value_list[13])])
    except:
        # Return NaN values if there's an error
        return pd.Series([float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan')])

num_files_dqn = len([name for name in os.listdir(input_dir_state_dqn) if os.path.isfile(os.path.join(input_dir_state_dqn, name))])
num_files_ppo = len([name for name in os.listdir(input_dir_state_ppo) if os.path.isfile(os.path.join(input_dir_state_ppo, name))])

if (num_files_dqn != num_files_ppo):
    print("The files is not equal, Please retest to equal episode.")
else:
    num_files = num_files_dqn
    filename = f'transitions{num_files}_plot.csv'
    filepath_dqn = os.path.join(input_dir_state_dqn, filename)
    filepath_ppo = os.path.join(input_dir_state_ppo, filename)

    df_dqn = pd.read_csv(filepath_dqn)
    df_ppo = pd.read_csv(filepath_ppo)

    df_dqn[['x','y','dis','steer', 'ang_dist', 'dis_r_t', 'dis_r_o', 'dis_r_p','lane_l', 'lane_c', 'lane_r', 'box_detected', 'pedestrian_detected', 'robot_angle']] = df_dqn['State'].apply(string_to_floats)
    df_ppo[['x','y','dis','steer', 'ang_dist', 'dis_r_t', 'dis_r_o', 'dis_r_p','lane_l', 'lane_c', 'lane_r', 'box_detected', 'pedestrian_detected', 'robot_angle']] = df_ppo['State'].apply(string_to_floats)

    dqn_steer = round((df_dqn['steer'].astype(float) * 0.6),2)
    dqn_dist = df_dqn['dis_r_t'].reindex(index=df_dqn['dis_r_t'].index[:0:-1])
    dqn_tar_dist = 2.733 - (dqn_dist * 3.5)

    ppo_steer = round((df_ppo['steer'].astype(float) * 0.6),2)
    ppo_dist = df_ppo['dis_r_t'].reindex(index=df_ppo['dis_r_t'].index[:0:-1])
    ppo_tar_dist = 2.733 - (ppo_dist * 3.5) 
    # dqn_dist = round(dqn_adist / ((2 * math.pi)),2)
    # ppo_dist = round(ppo_adist / ((2 * math.pi)),2)

    plt.figure(figsize=(16,8))
    fig,(ax1,ax2) = plt.subplots(1,2)
    ax1.plot(dqn_steer,'tab:orange', marker='o', markersize='2', label = 'DQN')
    ax1.plot(ppo_steer,'tab:blue', marker='+', markersize='2', label = 'PPO')
    ax1.set_title("Steer")
    ax1.legend()
    ax2.plot(dqn_tar_dist,'tab:orange',marker='o', markersize='2', label = 'DQN')
    ax2.plot(ppo_tar_dist,'tab:blue', marker='+', markersize='2', label = 'PPO')
    ax2.set_title("Distance")
    ax2.legend()
    ax1.set(xlabel='Steps',ylabel='Steer Angle (rad)')
    ax2.set(xlabel='Steps',ylabel='Robot Distance (meters)')
    fig.suptitle(f'Robot Steer Angle and Distance at {num_files}th episode')
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.3, top=0.85)
    plt.savefig(os.path.join(output_dir,f'steer_angle_episode{num_files}.png'))
    plt.clf()
    plt.close()